In [1]:
import os

from wndcharm.FeatureSpace import FeatureSpace
f=FeatureSpace.NewFromFileOfFiles( os.path.expanduser('data/medial_7_6_8_delta_bin.fof'), long=True )
f.ToFitFile('data/medial_7_6_8_delta_bin.fit')

LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 grp=0 seq=0 fs_col=0>
LOADED  <FeatureVector "T2_map_Resliced_Slice6-l.sig" label="0" n_features=2919 grp=0 seq=0 fs_col=1>
LOADED  <FeatureVector "T2_map_Resliced_Slice8-l.sig" label="0" n_features=2919 grp=0 seq=0 fs_col=2>
LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 grp=1 seq=0 fs_col=0>
LOADED  <FeatureVector "T2_map_Resliced_Slice6-l.sig" label="0" n_features=2919 grp=1 seq=0 fs_col=1>
LOADED  <FeatureVector "T2_map_Resliced_Slice8-l.sig" label="0" n_features=2919 grp=1 seq=0 fs_col=2>
LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 grp=2 seq=0 fs_col=0>
LOADED  <FeatureVector "T2_map_Resliced_Slice6-l.sig" label="0" n_features=2919 grp=2 seq=0 fs_col=1>
LOADED  <FeatureVector "T2_map_Resliced_Slice8-l.sig" label="0" n_features=2919 grp=2 seq=0 fs_col=2>
LOADED  <FeatureVector "T2_map_Resliced_Slice7-l.sig" label="0" n_features=2919 gr

In [2]:
dmat = f.ContiguousDataMatrix()
print (dmat)
print (f.data_list)
print (f._contiguous_ground_truth_labels)
print (f._contiguous_ground_truth_values)

[[  2.46300000e+03   5.44220000e-02   2.34303000e-01 ...,  -4.20041000e+04
    1.33342000e+05   7.16966000e-01]
 [  2.46100000e+03   3.82510000e-02   4.73540000e-02 ...,  -5.46250000e+04
    1.36239000e+05   7.25326000e-01]
 [  2.46300000e+03   2.86680000e-02   2.27273000e-01 ...,  -3.12424000e+04
    1.15157000e+05   7.13205000e-01]
 ..., 
 [  2.40200000e+03   7.63360000e-02   1.62866000e-01 ...,  -6.03631000e+04
    1.30804000e+05   7.21693000e-01]
 [  2.45000000e+03   1.28302000e-01   7.62710000e-02 ...,  -5.37427000e+04
    1.35765000e+05   7.17335000e-01]
 [  2.43000000e+03   1.49250000e-02   4.86660000e-02 ...,  -4.15241000e+04
    1.06458000e+05   7.17794000e-01]]
[array([[  2.46300000e+03,   5.44220000e-02,   2.34303000e-01, ...,
         -4.20041000e+04,   1.33342000e+05,   7.16966000e-01],
       [  2.46100000e+03,   3.82510000e-02,   4.73540000e-02, ...,
         -5.46250000e+04,   1.36239000e+05,   7.25326000e-01],
       [  2.46300000e+03,   2.86680000e-02,   2.27273000e-0

In [3]:
print (f.class_names)
class_vals = [float(x) for x in f.class_names]
print (class_vals)

['0', '1', '3', '2', '4']
[0.0, 1.0, 3.0, 2.0, 4.0]


In [4]:
import numpy as np
import scipy as sp
from scipy.stats import pearsonr
from sklearn.lda import LDA

In [5]:
def normalize (train, test):
    norm_train_set = train.copy() 
    mins, maxs = normalize_by_columns (norm_train_set)
    norm_test_set = test.copy() 
    normalize_by_columns (norm_test_set, mins, maxs)
    return (norm_train_set, norm_test_set)

def normalize_by_columns ( full_stack, mins = None, maxs = None ):
    """This is a global function to normalize a matrix by columns.
    If numpy 1D arrays of mins and maxs are provided, the matrix will be normalized against these ranges
    Otherwise, the mins and maxs will be determined from the matrix, and the matrix will be normalized
    against itself. The mins and maxs will be returned as a tuple.
    Out of range matrix values will be clipped to min and max (including +/- INF)
    zero-range columns will be set to 0.
    NANs in the columns will be set to 0.
    The normalized output range is hard-coded to 0-100
    """
    # Edge cases to deal with:
    # Range determination:
    # 1. features that are nan, inf, -inf
    # max and min determination must ignore invalid numbers
    # nan -> 0, inf -> max, -inf -> min
    # Normalization:
    # 2. feature values outside of range
    # values clipped to range (-inf to min -> min, max to inf -> max) - leaves nan as nan
    # 3. feature ranges that are 0 result in nan feature values
    # 4. all nan feature values set to 0

    # Turn off numpy warnings, since we're taking care of invalid values explicitly
    oldsettings = np.seterr(all='ignore')
    if (mins is None or maxs is None):
        # mask out NANs and +/-INFs to compute min/max
        full_stack_m = np.ma.masked_invalid (full_stack, copy=False)
        maxs = full_stack_m.max (axis=0)
        mins = full_stack_m.min (axis=0)

    # clip the values to the min-max range (NANs are left, but +/- INFs are taken care of)
    full_stack.clip (mins, maxs, full_stack)
    # remake a mask to account for NANs and divide-by-zero from max == min
    full_stack_m = np.ma.masked_invalid (full_stack, copy=False)

    # Normalize
    full_stack_m -= mins
    full_stack_m /= (maxs - mins)
    # Left over NANs and divide-by-zero from max == min become 0
    # Note the deep copy to change the numpy parameter in-place.
    full_stack[:] = full_stack_m.filled (0) * 100.0

    # return settings to original
    np.seterr(**oldsettings)

    return (mins,maxs)

In [6]:
def stand (train, test):
    scaler = StandardScaler()
    new_train_set = scaler.fit_transform(train)
    new_test_set = scaler.transform(test)
    return (new_train_set,new_test_set)

In [7]:
def round_robin_iteration (index, data_matrix_list):
    '''Does a leave N out, where N is the number of classes.
    The class with the smallest number of samples -1 (nsamples - 1) determines training set size.
    Picks nsamples-1 for training and testing from a circular list starting at index.
    Index ranges from 0 to the product of number of samples in each class.
    data_matrix_list is a list of data matrixes, with one matrix per class'''
    lengths = [m.shape[0] for m in data_matrix_list]
    nclasses = len(lengths)
    max_samples = min (lengths) - 1
    indexes = [0] * nclasses
    cl_index = index
    for i in range (nclasses-1,0,-1):
        indexes[i] = cl_index / lengths[i]
        cl_index -= (indexes[i] * lengths[i])
    indexes[0] = cl_index
    indexes = list(reversed(indexes))
    
    train_mats = []
    test_mats = []
    for class_num in range(nclasses):
        class_indexes = [ (count+indexes[class_num]+1) % lengths[class_num] for count in range (max_samples) ]
        train_mats.append (np.take (data_matrix_list[class_num], class_indexes, axis=0) )
        test_mats.append (np.take (data_matrix_list[class_num], [indexes[class_num]], axis=0) )
    return (train_mats, test_mats)

In [8]:
dat_mat = np.mgrid[0:8,0:5]
print (dat_mat)
(train,test) = round_robin_iteration (19,dat_mat)
print ('-------')
print (test)
print (train)

[[[0 0 0 0 0]
  [1 1 1 1 1]
  [2 2 2 2 2]
  [3 3 3 3 3]
  [4 4 4 4 4]
  [5 5 5 5 5]
  [6 6 6 6 6]
  [7 7 7 7 7]]

 [[0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]
  [0 1 2 3 4]]]
-------
[array([[2, 2, 2, 2, 2]]), array([[0, 1, 2, 3, 4]])]
[array([[3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4],
       [5, 5, 5, 5, 5],
       [6, 6, 6, 6, 6],
       [7, 7, 7, 7, 7],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1]]), array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])]


In [9]:
def list_to_contig_mat (data_matrix_list, class_vals):
    data_mat_contig = np.vstack (data_matrix_list)
    class_vals_vec_list = []
    for class_idx in range (len(data_matrix_list)):
        class_vals_vec_list += [class_vals[class_idx]] * len (data_matrix_list[class_idx])
    class_vals_contig = np.asarray(class_vals_vec_list)
    return (data_mat_contig,class_vals_contig)

In [10]:
def Fisher(split):
    """Takes a FeatureSet_Discrete as input and calculates a Fisher score for
    each feature. Returns a newly instantiated instance of FisherFeatureWeights.

    For:
    N = number of classes
    F = number of features
    It = total number of images in training set
    Ic = number of images in a given class
    """

    if split == None:
        import inspect
        form_str = 'You passed in a None as a training set to the function {0}.{1}'	
        raise ValueError( form_str.format( cls.__name__, inspect.stack()[1][3] ) )

    # we deal with NANs/INFs separately, so turn off numpy warnings about invalid floats.
    oldsettings = np.seterr(all='ignore')

    def get_train_3d (self):
        if self.train_3d is None:
            self.train_3d = get_class_mat_list (self.train_set, self.train_classed_labels)
        return (self.train_3d)

    def get_test_3d (self):
        if self.test_3d is None:
            self.test_3d = get_class_mat_list (self.test_set, self.test_classed_labels)
        return (self.test_3d)

    #class_mats = split.get_train_3d()
    class_mats = get_class_mat_list (split.train_set, split.train_classed_labels)
    # 1D matrix 1 * F
    population_means = np.mean( split.train_set, axis = 0 )
    n_classes = class_mats.shape[0]
    n_features = split.train_set.shape[1]

    # 2D matrix shape N * F
    intra_class_means = np.empty( [n_classes, n_features] )
    # 2D matrix shape N * F
    intra_class_variances = np.empty( [n_classes, n_features] )

    class_index = 0
    for class_feature_matrix in class_mats:
        intra_class_means[ class_index ] = np.mean( class_feature_matrix, axis=0 )
    # Note that by default, numpy divides by N instead of the more common N-1, hence ddof=1.
        intra_class_variances[ class_index ] = np.var( class_feature_matrix, axis=0, ddof=1 )
        class_index += 1

    # 1D matrix 1 * F
    # we deal with NANs/INFs separately, so turn off numpy warnings about invalid floats.
    # for the record, in numpy:
    # 1./0. = inf, 0./inf = 0., 1./inf = 0. inf/0. = inf, inf/inf = nan
    # 0./0. = nan, nan/0. = nan, 0/nan = nan, nan/nan = nan, nan/inf = nan, inf/nan = nan
    # We can't deal with NANs only, must also deal with pos/neg infs
    # The masked array allows for dealing with "invalid" floats, which includes nan and +/-inf
    denom = np.mean( intra_class_variances, axis = 0 )
    denom[denom == 0] = np.nan
    feature_weights_m = np.ma.masked_invalid (
            ( np.square( population_means - intra_class_means ).sum( axis = 0 ) /
        (n_classes - 1) ) / denom
        )
    # return numpy error settings to original
    np.seterr(**oldsettings)

    # the filled(0) method of the masked array sets all nan and infs to 0
    fisher_values = feature_weights_m.filled(0).tolist()

    return (fisher_values)

In [11]:
def Pearson(train_mat, train_vals):
    """Calculate regression parameters and correlation statistics that fully define
    a continuous classifier.

    At present the feature weights are proportional the Pearson correlation coefficient
    for each given feature."""

    from scipy import stats

    # Known issue: running stats.linregress() with np.seterr (all='raise') has caused
    # arithmetic underflow (FloatingPointError: 'underflow encountered in stdtr' )
    # I think this is something we can safely ignore in this function, and return settings
    # back to normal at the end. -CEC
    np.seterr (under='ignore')    

    pearson_coeffs = np.zeros(train_mat.shape[1])

    for feature_index in range( train_mat.shape[1] ):
        slope, intercept, pearson_coeff, p_value, std_err = stats.linregress(
            train_vals, train_mat[:,feature_index]
        )

        pearson_coeffs[feature_index] = pearson_coeff
# We're just returning the pearsons^2 now...
#    pearson_values = [val*val / r_val_squared_sum for val in pearson_coeffs ]
#    pearson_coeffs = (pearson_coeffs * pearson_coeffs) / r_val_squared_sum
    pearson_coeffs *= pearson_coeffs
    

    # Reset numpy
    np.seterr (all='raise')

    return pearson_coeffs

In [13]:
def sort_by_weight (the_mat, feature_weights):
    i = np.argsort(feature_weights)
    sort_mat = the_mat[:,i]
    sort_mat = np.fliplr(sort_mat)
    return (sort_mat)

def weigh_sort(train, test, feature_weights):
    weigh_train = np.multiply (norm_train, feature_weights)
    weigh_test = np.multiply (norm_test, feature_weights)

    sorted_train = sort_by_weight (weigh_train, feature_weights)
    sorted_test = sort_by_weight (weigh_test, feature_weights)
    return (sorted_train, sorted_test)

In [14]:
nfeatures = 200
niter = np.product ( [m.shape[0] for m in f.data_list] )
class_vals = [float(x) for x in f.class_names]
n_correct = np.asarray( [0]*2)
(train,test) = round_robin_iteration (0,f.data_list)
print ('Train class sizes : {}'.format([x.shape[0] for x in train]))
print ('Features          : {}'.format(nfeatures))
print ('Iterations        : {}'.format(niter))
for iter_idx in range ( niter ):
    # Split
    (train,test) = round_robin_iteration (iter_idx,f.data_list)
    (contig_train_mat, contig_train_vals) = list_to_contig_mat (train, class_vals)
    (contig_test_mat, contig_test_vals) = list_to_contig_mat (test, class_vals)

    # Normalize
    (norm_train, norm_test) = normalize (contig_train_mat, contig_test_mat)
    
    # Reduce
    feature_weights = Pearson(norm_train, contig_train_vals)
    (sorted_train, sorted_test) = weigh_sort (norm_train, norm_test, feature_weights)

    # Classify
    preds,pred_vals = WND5(sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)

#    preds = rand_forest_clf (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals, iter_idx)
#    preds = rand_forest_clf (sorted_train[:,:nfeatures], np.asarray([norm_test[1][:,:nfeatures],sorted_test[0][:,:nfeatures]]), contig_train_vals, iter_idx)
    for pred_idx in range (len(preds)):
        if (preds[pred_idx] == contig_test_vals[pred_idx]):
            n_correct[pred_idx] += 1
    cumul_acc = [(float(x) / (float(iter_idx)+1.0)) for x in n_correct]
    print ('Iteration {}; Predictions: {}; cumulative accuracies: {}'.format(iter_idx, preds, cumul_acc))

Train class sizes : [2, 2, 2, 2, 2]
Features          : 200
Iterations        : 145152


NameError: name 'WND5' is not defined

In [ ]:
dmat = f.ContiguousDataMatrix()
print (dmat)
print (f.data_list)
print (f._contiguous_ground_truth_labels)
print (f._contiguous_ground_truth_values)

In [ ]:
iter_idx = 1

print dmat
dmat2 = np.delete (dmat,[iter_idx],axis=0)
print
print np.asarray([dmat2[iter_idx]])

class_vals = f._contiguous_ground_truth_values
print class_vals
contig_train_vals = np.delete (class_vals, [iter_idx])
contig_test_vals = np.asarray ([class_vals[iter_idx]])
print contig_train_vals
print contig_test_vals